<a href="https://colab.research.google.com/github/skywalker0803r/Ricky/blob/master/ensemble_submit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

df_1 = pd.read_csv('/content/drive/My Drive/玉山人工智慧比賽數據/lightgbm_result/submission_lightgbm_Fold_0.csv',index_col=0)
df_2 = pd.read_csv('/content/drive/My Drive/玉山人工智慧比賽數據/lightgbm_result/submission_lightgbm_Fold_1.csv',index_col=0)
df_3 = pd.read_csv('/content/drive/My Drive/玉山人工智慧比賽數據/lightgbm_result/submission_lightgbm_Fold_2.csv',index_col=0)
df_4 = pd.read_csv('/content/drive/My Drive/玉山人工智慧比賽數據/lightgbm_result/submission_lightgbm_Fold_3.csv',index_col=0)
df_5 = pd.read_csv('/content/drive/My Drive/玉山人工智慧比賽數據/lightgbm_result/submission_lightgbm_Fold_4.csv',index_col=0)

In [3]:
final = pd.DataFrame()

final['txkey'] = df_1['txkey']

final['pred_1'] = df_1['fraud_ind'].values
final['pred_2'] = df_2['fraud_ind'].values
final['pred_3'] = df_3['fraud_ind'].values
final['pred_4'] = df_4['fraud_ind'].values
final['pred_5'] = df_5['fraud_ind'].values

final['fraud_ind'] = final.drop('txkey',axis=1).mean(axis=1)

final = final[['txkey','fraud_ind']]

final.head()

,txkey,fraud_ind
1521787,592489,0.0
1521788,592452,0.0
1521789,590212,0.0
1521790,590209,0.0
1521791,592488,0.0


In [4]:
final['fraud_ind'].unique()

array([0. , 0.4, 0.8, 0.2, 0.6, 1. ])

In [0]:
def f(x):
  if x>=0.4:
    return 1
  else:
    return 0
final['fraud_ind'] = final['fraud_ind'].apply(f)

In [6]:
ps={}

for index,data in enumerate([df_1,df_2,df_3,df_4,df_5,final]):
  ps['p'+str(index)] = round((data['fraud_ind'].value_counts()[1] / #盜刷樣本數
                len(data['fraud_ind']))*100 ,5) #總數

pd.DataFrame(ps,index=['盜刷比例%']).rename(columns={'p5':'final_p'})

,p0,p1,p2,p3,p4,final_p
盜刷比例%,1.11866,1.09945,1.13147,1.09376,1.18506,1.27827


# 保存預測結果

In [0]:
path = "/content/drive/My Drive/玉山人工智慧比賽數據/lightgbm_result/submission_Lightgbm_Voting_big_than_0_4_result.csv"
final.to_csv(path)

In [8]:
print(final["fraud_ind"].value_counts())

0    416275
1      5390
Name: fraud_ind, dtype: int64
